Quick code to capture video, courtesy of opencv documentation
This is becoming more of a platform, worth cleaning up 

Todo: 
Get audio working - X
Get FPS for Video - X
Synch up audio and video - This is a problem...possibly unsurmountable? 
Use some functions you philistine - X
Clean up UI
Move off jupyter: debug while we're at it
It might be better to have openCV stop while recording, for simplicity
-Yes, but it's nice to have it running, for Science

In [2]:
# Quick code to capture video, courtesy of opencv documentation
# This is becoming more of a platform, worth cleaning up 
# 
# Todo: 
# Synch up audio and video - This is a problem...possibly unsurmountable? 
# Clean up UI
# Add debugging
# Add Menu? - Set parameters, devices, bounding box, etc 
# Specify Audio output device - x
# Play sound from file - x
# Order sound playback (random? Set? Maybe based on date? 

import numpy as np
import cv2
import subprocess
import sys,os
from subprocess import Popen, PIPE
from PIL import Image
import time, datetime 
import threading
import pyaudio, wave

RECORDING = False
    
def monitor_cam(cam_count = 1):
    caps = [0] * cam_count
    counts = [0] * cam_count
    clips = [[]] * cam_count
    rets = [0] * cam_count
    frames = [0] * cam_count
    grays = [0] * cam_count
    subgrays = [0] * cam_count
    subgrays2 = [0] * cam_count
    for n in range(cam_count):
        cam_name = 'Cam' + str(n)
        cv2.namedWindow(cam_name)
        caps[n] = cv2.VideoCapture(n)
    
    CONT = True
    while(CONT):
        for n in range(cam_count):
            rets[n],frames[n] = caps[n].read()
            grays[n] = cv2.cvtColor(frames[n], cv2.COLOR_BGR2GRAY)
            
            # Mark the bounding box(es)
            # this is probably horribly inefficient...
            grays[n][99,1000:1100] = 0
            grays[n][201,1000:1100] = 0
            grays[n][100:200,999] = 0
            grays[n][100:200,1101] = 0
            
            grays[n][99,100:200] = 0
            grays[n][201,100:200] = 0
            grays[n][100:200,99] = 0
            grays[n][100:200,201] = 0
            
            subgrays[n] = grays[n][100:200,1000:1100]
            subgrays2[n] = grays[n][100:200,100:200]
            if subgrays[n].mean() < 100:
                gray1 = True
                grays[n][99,1000:1100] = 255
                grays[n][201,1000:1100] = 255
                grays[n][100:200,999] = 255
                grays[n][100:200,1101] = 255
            else:
                gray1 = False
            if  subgrays2[n].mean() < 100:
                gray2 = True
                grays[n][99,100:200] = 255
                grays[n][201,100:200] = 255
                grays[n][100:200,99] = 255
                grays[n][100:200,201] = 255
            else:
                gray2 = False
            if gray1 == True and gray2 == True:
                counts[n] += 1
                if counts[n] >= 50 and RECORDING == False:     
                    #Add a ten second record function
                    RECORDING == True
                    record_AV(caps[n])
                    counts[n] = 0
                    
                elif RECORDING == True:
                    counts[n] = 0
            else:
                count = 0
            cv2.imshow('Cam'+ str(n),grays[n])
            if cv2.waitKey(100) & 0xFF == ord('q'):
                CONT = False
                
    for n in range(cam_count):
        caps[n].release()
    cv2.destroyAllWindows()
    return

## Wait some set amount and says a string
def speak(phrase = "Hi there", wait = 2):
    time.sleep(wait)
    subprocess.call(["say",phrase])
    return

## Play some .wav from a file using a specified output device
def play_wav(p_audio = pyaudio.PyAudio, wav_file = "test.wav", out_index = 1):
    CHUNK = 1024
    p = p_audio
    wf = wave.open(wav_file, 'rb')
    stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                    channels=wf.getnchannels(),
                    rate=wf.getframerate(),
                    output=True, 
                    output_device_index = out_index)
    data = wf.readframes(CHUNK)
    while len(data) > 0:
        stream.write(data)
        data = wf.readframes(CHUNK)

    stream.stop_stream()
    stream.close()
    return

def record_AV(cap, mic = None):
    # Start Audio & Video, then play the sound
    #print('booting up threads')
    t = threading.Thread(target=speak)
    #t = threading.Thread(target=play_wav,args(out_index=mic,) 
    #record_audio()
    a = threading.Thread(target=record_audio, args=(10,mic,))
    #record_video()
    v = threading.Thread(target=record_video, args=(cap,))
    
    print('starting audio')
    a.start()
   
    #print('starting video')
    v.start() 
    
    #print('playing sound')
    t.start()
    """
    count = 0
    # Add a ~10s monitor so you see the posture
    while(count<100):
        ret0,frame0 = cap0.read()
        ret1,frame1 = cap1.read()
        
        gray0 = cv2.cvtColor(frame0,cv2.COLOR_BGR2GRAY)
        gray1 = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)
        cv2.imshow('cam0',gray0)
        cv2.imshow('cam1',gray1)
        count += 1
        cv2.waitKey(5)
    """
    #The join stops it from continueing until it's done..
    #a.join()
    #v.join()
    #t.join()
    
    return 

# Record and save audio
# Defaults to default microphone, you'll need to know the specific index otherwise
# There could be a way to automate it...
def record_audio(length = 10,mic = None):
    
    #print('recording audio...')
    p = pyaudio.PyAudio()
    
    if mic == None:
        dev_index = 0
    else:
        dev_index = mic
    device_info = p.get_device_info_by_index(dev_index)
    
    # Set Parameters
    FRAMES_PERBUFF = 2048
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    FRAME_RATE = int(device_info['defaultSampleRate'])
    # Boot up stream and populate data then close stream (I'll know the amount of time recording)
    stream = p.open(format=FORMAT,channels=CHANNELS,rate=FRAME_RATE,input=True, input_device_index=mic,frames_per_buffer=FRAMES_PERBUFF)
    frames = []

    REC_SECONDS = length
    nchunks = int(REC_SECONDS * FRAME_RATE / FRAMES_PERBUFF)
    a_start = time.time()
    for i in range(0, nchunks):
        data = stream.read(FRAMES_PERBUFF)
        frames.append(data)
    a_stop = time.time()
    stream.stop_stream()
    stream.close()

    #print('saving audio...')
    date_string = time.strftime("%Y-%m-%d-%H:%M")
    outfile = 'test_audio_' + date_string + ".wav"
    
    wf = wave.open(outfile,'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(FRAME_RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    global A_START
    global A_FINISH
    global A_TMP
    A_START = a_start
    A_FINISH = a_stop
    A_TMP = outfile
    return

def record_video(cap):
    print('recording video...')
    clip = []
    """
    v_0 = time.time()
    ret,frame = cap.read()
    clip.append(frame)
    v_start = time.time()
    cv2.waitKey(40)
    v_wait = time.time()
    ret,frame = cap.read()
    clip.append(frame)
    v_2 = time.time()
    
    print('time series:')
    print(v_0)
    print(v_start)
    print(v_wait)
    print(v_2)
    """
    
    v_start = time.time()
    
    vid_time = 10
    while time.time() <= v_start + vid_time:
        ret, frame = cap.read()
        clip.append(frame)
        cv2.waitKey(40)
    """    
    while(len(clip) <= 300):
        ret, frame = cap.read()
        clip.append(frame)  
        cv2.waitKey(40)
    """    
    v_stop = time.time()
    vid_length = v_stop - v_start
    num_frames = len(clip)
    fps = num_frames / vid_length
    print(fps)
    round_fps = int(round(fps))
    global V_START
    global V_FINISH
    
    V_START = v_start
    V_FINISH = v_stop
    
    save_clip(clip,fps)

    return

def save_clip(clip,fps):
    global RECORDING
    print('saving clip....')
    date_string = time.strftime("%Y-%m-%d-%H:%M:%S")
    outfile = 'test_vid' + date_string + ".avi"
    framerate = str(fps)
    p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'mjpeg', 
               '-r', framerate, '-i', '-', '-vcodec', 'mpeg4', '-qscale', 
               '5', '-r', framerate, outfile], stdin=PIPE)
    fps, duration = 20, len(clip) / 20.
    for i in range(len(clip)):
        col_im = cv2.cvtColor(clip[i],cv2.COLOR_BGR2RGB)
        rot_im = np.rot90(col_im,3)
        im = Image.fromarray(rot_im)
        im.save(p.stdin, 'JPEG')
    #print(clip[i])
    p.stdin.close()
    p.wait()
    
    global A_START,V_START, A_TMP
    offset = V_START - A_START
    print(V_START)
    print(A_START)
    a_file = A_TMP
    splice_AV(a_file,outfile,offset)
    RECORDING = False
    print("monitoring...")
    return

# Combine audio & video with calculated offset
def splice_AV(a_file,v_file,offset):
    outfile = 'av_' + v_file
    
    # quickly convert offset to time

    p = Popen(['ffmpeg','-i',v_file, '-itsoffset', str(offset), 
           '-i', a_file, '-vcodec', 'copy', '-acodec', 'copy', outfile])
    p.wait()
    #Popen(['rm',a_file,v_file])
    return
    
if __name__ == '__main__':
    monitor_cam()


starting audio
recording video...
starting audio
recording video...
13.784816714659849
saving clip....
12.356336449372595
saving clip....
1552238457.784596
1552238457.788982
monitoring...
1552238457.784596
1552238457.788982
monitoring...


In [ ]:
## In theory, this should work on a windows, but doesn't work on a mac

height,width = clip[1].shape
fourcc = cv2.cv.CV_FOURCC('M','J','P','G')
video = cv2.VideoWriter('test.mpg',fourcc,10,(width,height),0)
#write the video, reset the variables, leave the loop
for i in range(len(clip)):
    clip_color = cv2.cvtColor(clip[i], cv2.cv.CV_GRAY2RGB)
    video.write(clip_color)
video.release()

In [ ]:
Popen(['ffmpeg','-i','my_video.avi', '-itsoffset', str(d), 
       '-i', 'my_audio.wav', '-vcodec', 'copy', '-acodec', 'copy', 'output2.avi'])

In [ ]:
offset = V_START - A_START
print offset
print str(-offset)

In [ ]:
s, ms = divmod(offset,1)
m, s = divmod(offset, 60)
h, m = divmod(m, 60)
dt_offset = "%02d:%02d:%f" % (h, m, offset)

dt_offset

In [2]:
import pyaudio, wave

p = pyaudio.PyAudio()
mic = 0
if mic == None:
    dev_index = 0
else:
    dev_index = mic
device_info = p.get_device_info_by_index(dev_index)
print int(device_info['defaultSampleRate'])

44100


In [3]:
p.get_default_output_device_info()

{'defaultHighInputLatency': 0.1,
 'defaultHighOutputLatency': 0.014036281179138322,
 'defaultLowInputLatency': 0.01,
 'defaultLowOutputLatency': 0.0038775510204081634,
 'defaultSampleRate': 44100.0,
 'hostApi': 0L,
 'index': 1L,
 'maxInputChannels': 0L,
 'maxOutputChannels': 2L,
 'name': u'Built-in Output',
 'structVersion': 2L}

In [6]:
import pyaudio
p = pyaudio.PyAudio()

mic = 0
# Get some info if you need it

def_info = p.get_default_input_device_info()
host_info = p.get_default_host_api_info()
device_info = p.get_device_info_by_index(0)

# Set Parameters
FRAMES_PERBUFF = 2048
FORMAT = pyaudio.paInt16
CHANNELS = 1
FRAME_RATE = int(device_info['defaultSampleRate'])

# Boot up stream and populate data then close stream (I'll know the amount of time recording)
stream = p.open(format=FORMAT,channels=CHANNELS,rate=FRAME_RATE,input=True,input_device_index=mic,frames_per_buffer=FRAMES_PERBUFF)
frames = []

REC_SECONDS = 5
nchunks = int(REC_SECONDS * FRAME_RATE / FRAMES_PERBUFF)
for i in range(0, nchunks):
    data = stream.read(FRAMES_PERBUFF)
    frames.append(data)
stream.stop_stream()
stream.close()



In [5]:
# Currently 
# 0 : Default Mic
# 1 : Default Output
# 2 : HDMI (Input/Output)
# 3 : Cable Creation (Input/Output)
import pyaudio
p = pyaudio.PyAudio()
p.get_device_info_by_index(3)





{'defaultHighInputLatency': 0.1,
 'defaultHighOutputLatency': 0.0126875,
 'defaultLowInputLatency': 0.01,
 'defaultLowOutputLatency': 0.0033541666666666668,
 'defaultSampleRate': 48000.0,
 'hostApi': 0L,
 'index': 3,
 'maxInputChannels': 1L,
 'maxOutputChannels': 2L,
 'name': u'CableCreation',
 'structVersion': 2L}

In [9]:
out_stream = p.open(format = pyaudio.paInt16, channels=1, rate=44100, output=True, output_device_index = 1)

for i in frames:
    out_stream.write(i)

In [ ]:
# Save the file
import wave
wf = wave.open('test_auido.wav','wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(FRAME_RATE)
wf.writeframes(b''.join(frames))
wf.close()

In [ ]:
# My first thread practice
import threading 
import logging
import time

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.basicConfig(level=logging.DEBUG,
                   format='[%(levelname)s] (%(threadName)-10s) %(message)s',
                   )
def worker(id = "Unknown"):
    logging.debug('Starting')
    print "Worker number: " + str(id)
    time.sleep(2)
    logging.debug('Exiting')
    return 0 

def my_service():
    logging.debug('Starting')
    time.sleep(3)
    logging.debug('Exiting')
    
t = threading.Thread(name='my_service',target=my_service)
w = threading.Thread(name='Worker 1',target=worker,args=(1,))
w2 = threading.Thread(target=worker)

w.start()
w2.start()
t.start()

In [ ]:
## Making a Daemon...
# So a non-daemon still runs concurrently, it's just that the program has to wait until it's finished. 
# This distinction so far doesn't not seem critical in jupyter, but it does allow me to keep going. Which is rad
def daemon():
    logging.debug('daemon Starting')
    time.sleep(5)
    print 'I am many'
    logging.debug('daemon Exiting')
    
d=threading.Thread(name='daemon', target=daemon)
d.setDaemon(True)

def non_daemon():
    logging.debug('non daemon starting')
    logging.debug('non daemon Exiting')
    
t = threading.Thread(name='non-daemon', target=non_daemon)

d.start()
t.start()

print 'all done'


In [ ]:
import threading
import time

def evens():
    for e in range(10):
        print str(2*e)
        time.sleep(2)
    return 1

def odds():
    for o in range(10):
        print str(2*e -1)
        time.sleep(1)
    return 1

e = threading.Thread(target=evens)
o = threading.Thread(target=odds)

e.start
o.start

while evens() != 1:
    print "I'm waiting..."
    time.sleep(3)
    
for i in range(5):
    print "I'm running ahead!"

In [ ]:
e = threading.Thread(target=evens)


e.start()
e.join()


print "Very Done!"